<a href="https://colab.research.google.com/github/EvgenyEsin/Spark_Apache/blob/main/Spark_Apache_sem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ к семинару 3
Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=eda8b0d30f605e20aa4861b4803b114449e5f8ea1a80dee4edf11f17559981be
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

In [4]:
# Создание Spark сессии
spark = SparkSession.builder.appName("Homework4").getOrCreate()

In [5]:
# Исходные данные
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]

In [6]:
# Создание DataFrame
schema = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, schema)

In [7]:
# Преобразование поля date в тип данных DateType
df = df.withColumn("date", fn.to_date("date"))

In [8]:
# Определение оконной спецификации
window_spec = Window.partitionBy("category")

In [9]:
# Оконная функция: вычисление средней выручки от продаж для каждой категории продукта
avg_revenue = fn.avg("revenue").over(window_spec)

In [10]:
# Применение оконной функции и приведение к double
result_df = df.withColumn("avg_revenue", avg_revenue.cast(DoubleType()))

In [11]:
# Отображение результатов: используем distinct() для исключения повторов
result_df.select(["category", "avg_revenue"]).distinct().show()

+-----------+------------------+
|   category|       avg_revenue|
+-----------+------------------+
|      Books|10333.333333333334|
|   Clothing|           15000.0|
|Electronics|           12500.0|
+-----------+------------------+



In [12]:
# Определение при помощи функции pivot средних значений выручки от продаж по категориям за соответствующий день
pivot_df = df.groupBy("category").pivot("date").agg(fn.avg("revenue")).orderBy("category")

# Вывод результатов
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+

